In [ ]:
import json
import os.path
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import functools
import matplotlib.pyplot as plt
from typing_extensions import TypedDict
from typing import List

In [ ]:
# Config

useDataOverYears = 5 # 1y,2y,5y,10y
endDate = datetime.date.today() # only influences parsing atm, not collection
portfolioFile = 'portfolio.json'

In [ ]:
# Types

class EquityItem(TypedDict):
    ticker: str
    name: str
    amount: int
        
class OtherItem(TypedDict):
    name: str
    amount: int
    fromDate: any
    toDate: any
    changePerAnnum: float
        
class Portfolio(TypedDict):
    equities: List[EquityItem]
    other: List[OtherItem]
        
class PortfolioInfo(TypedDict):
    name: str
    fromDate: any
    toDate: any
    changePerAnnum: float

In [ ]:
# Load and Initialise Portfolio
def loadPortfolio() -> Portfolio:
    sample = 'portfolioSample.json'
    useSample = False
    if not os.path.isfile(portfolioFile):
        print("Using sample data.\nPlease create a portfolio.json to use custom input.")
        useSample = True
    with open(sample if useSample else portfolioFile, 'r') as infile:
        return json.load(infile)

portfolio = loadPortfolio()
portfolioEquities = portfolio["equities"]
portfolioOther = portfolio["other"]

In [ ]:
# Download equities

def getDataFileName(name: str) -> str:
    return f'data/{name}.json'

def convertDataFrameToJson(df: pd.DataFrame) -> any:
    return json.loads(df.to_json())

def writeJsonToFile(name: str, payload: any) -> None:
    filename = getDataFileName(name)
    with open(filename, 'w') as outfile:
        json.dump(payload, outfile)

    print(f"{filename} saved!")
    
def getYFDataFrame(ticker: str, years: int) -> pd.DataFrame:
    dataframe = yf.download(
        ticker, 
        period=str(years)+'y',
        prepost=True,
        interval="1d" # none of the larger intervals seem to be consistent
    )
    
    return dataframe

def downloadPortfolioEquities(equities: List[EquityItem], years: int, partial=True) -> None:
    for stock in equities:
        ticker = stock["ticker"]
        if not os.path.isfile(getDataFileName(ticker)) or not partial:
            print(f"Retreiving {ticker} information...")
            writeJsonToFile(
                ticker,
                convertDataFrameToJson(
                    getYFDataFrame(ticker, years)
                )
            )
        else:
            print(f"{ticker} already exists. Skipping download")

    
downloadPortfolioEquities(portfolioEquities, useDataOverYears)

In [ ]:
# Calculate equity means and produce weighted outputs

def getPortfolioWeight(name: str) -> float:
    allItems = []
    
    # flatten portfolio
    for key, value in portfolio.items():
        allItems.extend(value)
    
    portfolioSum = functools.reduce(lambda acc, curr: acc + curr['amount'], allItems, 0)
    itemVal = 0
    for item in allItems:
        if item['name'] == name:
            itemVal = item['amount']
            
    return itemVal/portfolioSum
        
def percentOfDifference(x):
    current = x.iloc[-1]
    previous = x.iloc[0]
    print(f'previous: {previous} ::: current {current}')
    
    return (current - previous)
    
def parseStockDataFrame(df: pd.DataFrame, portfolioEntry: EquityItem) -> PortfolioInfo:    
    # initialise fundamental dates
    minDate = df.index[0]
    maxDate = endDate - datetime.timedelta(weeks=useDataOverYears*52)
    fromDate = maxDate if maxDate >= minDate else minDate
    
    relevantDf = df[df.index > fromDate.isoformat()]
    
    closes = relevantDf["Adj Close"]
    differences = []
    # this is pretty inefficient for pandas, I'm aware
    # but I don't know how else we can get a rolling difference
    for i, day in enumerate(closes):
        previous = closes[i-1]
        current = closes[i]
        if i == 0:
            differences.append(0)
        elif not pd.isna(previous) and not pd.isna(current):
            diff = (current - previous) / previous
            differences.append(diff)
        else:
            raise Exception('NAN in data')
        
    changePerAnnum = np.mean(differences) * 253 # about ~253 working days in a year
    
    
    return {
        'name': portfolioEntry['name'],
        'weight': getPortfolioWeight(portfolioEntry['name']),
        'fromDate': fromDate,
        'toDate': endDate,
        'amount': portfolioEntry["amount"],
        'changePerAnnum': changePerAnnum
    }

portfolioEquityMeans = []
for entry in portfolioEquities:
    portfolioEquityMeans.append(
        parseStockDataFrame(
            pd.read_json(
                getDataFileName(entry['ticker'])
            ),
            entry
        )
    )

portfolioOtherMapped = []
for entry in portfolioOther:
    portfolioOtherMapped.append(
        {
            **entry,
            'weight': getPortfolioWeight(entry['name']),
        }
    )
    
portfolioDf = pd.concat([
    pd.DataFrame(portfolioEquityMeans), 
    pd.DataFrame(portfolioOtherMapped)
])
portfolioDf['weightedReturnPerAnnum'] = portfolioDf['weight'] * portfolioDf['changePerAnnum']
portfolioDf



In [ ]:
# Visualise results

labels = portfolioDf['name']
weights = portfolioDf['weight']
weightedChange = portfolioDf['weightedReturnPerAnnum']
colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue']

# Weights
fig = plt.figure(figsize=(6,6));
plt.pie(
    weights, 
    labels=labels, 
    colors=colors,
    autopct='%1.1f%%', 
    shadow=True, 
    startangle=140
)
plt.axis('equal')
plt.title("Portfolio Weights", pad=15)

plt.show()

# Change By Weight
fig = plt.figure(figsize=(10,5));
plt.bar(
    labels, 
    weightedChange * 100,
    0.5
)
plt.title("Portfolio Weighted Expected Return %", pad=15)

plt.show()
print(f"Total Expected % Return is {weightedChange.sum()*100}")